In [8]:
import gc

import modin.pandas as pd

import modin
import pandas

In [2]:
df = pd.read_csv("~/data/G1_1e7_1e2_0_0.csv")

To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.
Reason: No such file: '/Users/matthew.powers/Documents/code/my_apps/pydata-examples/modin/~/data/G1_1e7_1e2_0_0.csv'


In [3]:
df

,id1,id2,id3,id4,id5,id6,v1,v2,v3
0,id016,id016,id0000042202,15,24,5971,5,11,37.211254
1,id039,id045,id0000029558,40,49,39457,5,4,48.951141
2,id047,id023,id0000071286,68,20,74463,2,14,60.469241
3,id043,id057,id0000015141,32,43,63743,1,15,7.692145
4,id054,id040,id0000011083,9,25,16920,2,9,22.863525
...,...,...,...,...,...,...,...,...,...
9999995,id003,id044,id0000019084,86,65,81507,4,8,30.926485
9999996,id096,id068,id0000020362,4,85,95388,3,11,53.268709
9999997,id071,id006,id0000046057,52,57,55223,1,13,90.368393
9999998,id057,id099,id0000060922,47,36,90944,3,8,34.878200


In [4]:
df.head(3)

,id1,id2,id3,id4,id5,id6,v1,v2,v3
0,id016,id016,id0000042202,15,24,5971,5,11,37.211254
1,id039,id045,id0000029558,40,49,39457,5,4,48.951141
2,id047,id023,id0000071286,68,20,74463,2,14,60.469241


In [5]:
del df

## 1e8 CSV

In [9]:
%%time

modin.pandas.read_csv("~/data/G1_1e8_1e2_0_0.csv").groupby("id1").agg({"v1": "sum"})

CPU times: user 1min 6s, sys: 10.5 s, total: 1min 16s
Wall time: 1min 23s


,v1
id1,
id001,2997617
id002,2997194
id003,3001204
id004,3000945
id005,3001731
...,...
id096,2997704
id097,3004476
id098,2996139


In [7]:
%%time

pandas.read_csv("~/data/G1_1e8_1e2_0_0.csv").groupby("id1").agg({"v1": "sum"})

CPU times: user 42.8 s, sys: 10.7 s, total: 53.5 s
Wall time: 55.6 s


,v1
id1,
id001,2997617
id002,2997194
id003,3001204
id004,3000945
id005,3001731
...,...
id096,2997704
id097,3004476
id098,2996139


## 1e8 Parquet

In [10]:
%%time

modin.pandas.read_parquet("~/data/parquet/G1_1e8_1e2_0_0.parquet").groupby("id1").agg(
    {"v1": "sum"}
)

2023-07-14 17:33:29,483 - distributed.worker - WARNING - Could not find data: {'function-fd90521beb884726903b02ce296ec37c': ['tcp://127.0.0.1:60268', 'tcp://127.0.0.1:60284']} on workers: [] (who_has: {'function-fd90521beb884726903b02ce296ec37c': ['tcp://127.0.0.1:60268', 'tcp://127.0.0.1:60284']})
2023-07-14 17:33:29,484 - distributed.scheduler - WARNING - Worker tcp://127.0.0.1:60289 failed to acquire keys: {'function-fd90521beb884726903b02ce296ec37c': ('tcp://127.0.0.1:60268', 'tcp://127.0.0.1:60284')}


CPU times: user 2.16 s, sys: 1.27 s, total: 3.43 s
Wall time: 46.9 s


,v1
id1,
id001,2997617
id002,2997194
id003,3001204
id004,3000945
id005,3001731
...,...
id096,2997704
id097,3004476
id098,2996139


In [14]:
%%time

pandas.read_parquet("~/data/parquet/G1_1e8_1e2_0_0.parquet").groupby("id1").agg(
    {"v1": "sum"}
)

CPU times: user 24.5 s, sys: 4.54 s, total: 29.1 s
Wall time: 29.6 s


,v1
id1,
id001,2997617
id002,2997194
id003,3001204
id004,3000945
id005,3001731
...,...
id096,2997704
id097,3004476
id098,2996139


## 1e8 Parquet + column pruning

In [15]:
%%time

modin.pandas.read_parquet(
    "~/data/parquet/G1_1e8_1e2_0_0.parquet", columns=["id1", "v1"]
).groupby("id1").agg({"v1": "sum"})

CPU times: user 383 ms, sys: 106 ms, total: 490 ms
Wall time: 7.11 s


,v1
id1,
id001,2997617
id002,2997194
id003,3001204
id004,3000945
id005,3001731
...,...
id096,2997704
id097,3004476
id098,2996139


In [16]:
%%time

pandas.read_parquet(
    "~/data/parquet/G1_1e8_1e2_0_0.parquet", columns=["id1", "v1"]
).groupby("id1").agg({"v1": "sum"})

CPU times: user 4.64 s, sys: 731 ms, total: 5.37 s
Wall time: 5.23 s


,v1
id1,
id001,2997617
id002,2997194
id003,3001204
id004,3000945
id005,3001731
...,...
id096,2997704
id097,3004476
id098,2996139


## Does pandas CSV usecols make queries run faster?

In [17]:
%%time

modin.pandas.read_csv("~/data/G1_1e8_1e2_0_0.csv", usecols=["id1", "v1"]).groupby(
    "id1"
).agg({"v1": "sum"})

CPU times: user 19.4 s, sys: 3.03 s, total: 22.4 s
Wall time: 32.4 s


,v1
id1,
id001,2997617
id002,2997194
id003,3001204
id004,3000945
id005,3001731
...,...
id096,2997704
id097,3004476
id098,2996139


In [18]:
%%time

pandas.read_csv("~/data/G1_1e8_1e2_0_0.csv", usecols=["id1", "v1"]).groupby("id1").agg(
    {"v1": "sum"}
)

CPU times: user 18.2 s, sys: 2.47 s, total: 20.7 s
Wall time: 20.1 s


,v1
id1,
id001,2997617
id002,2997194
id003,3001204
id004,3000945
id005,3001731
...,...
id096,2997704
id097,3004476
id098,2996139


## 1e9 CSV + usecols

In [19]:
%%time

modin.pandas.read_csv("~/data/G1_1e9_1e2_0_0.csv", usecols=["id1", "v1"]).groupby(
    "id1"
).agg({"v1": "sum"})

2023-07-14 17:42:23,433 - distributed.scheduler - WARNING - Worker tcp://127.0.0.1:60280 failed to acquire keys: {'function-80b9df3d3be34586a24528ea30d85713': ('tcp://127.0.0.1:60269', 'tcp://127.0.0.1:60289', 'tcp://127.0.0.1:60277')}
2023-07-14 17:42:23,430 - distributed.worker - WARNING - Could not find data: {'function-80b9df3d3be34586a24528ea30d85713': ['tcp://127.0.0.1:60269', 'tcp://127.0.0.1:60289', 'tcp://127.0.0.1:60277']} on workers: [] (who_has: {'function-80b9df3d3be34586a24528ea30d85713': ['tcp://127.0.0.1:60269', 'tcp://127.0.0.1:60289', 'tcp://127.0.0.1:60277']})
2023-07-14 17:42:26,739 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 4.91 GiB -- Worker memory limit: 6.40 GiB
2023-07-14 17:42:27,263 - distributed.worker.memory - WARNING

ValueError: Sample larger than population or is negative

In [20]:
%%time

pandas.read_csv("~/data/G1_1e9_1e2_0_0.csv", usecols=["id1", "v1"]).groupby("id1").agg(
    {"v1": "sum"}
)

CPU times: user 3min 7s, sys: 25.9 s, total: 3min 33s
Wall time: 3min 29s


,v1
id1,
id001,30009448
id002,29996534
id003,30003365
id004,30015990
id005,29993888
...,...
id096,29993372
id097,30015928
id098,29997789


## 1e9 CSV + pyarrow + usecols

In [ ]:
%%time

modin.pandas.read_csv(
    "~/data/G1_1e9_1e2_0_0.csv", engine="pyarrow", usecols=["id1", "v1"]
).groupby("id1").agg({"v1": "sum"})

In [ ]:
%%time

pandas.read_csv(
    "~/data/G1_1e9_1e2_0_0.csv", engine="pyarrow", usecols=["id1", "v1"]
).groupby("id1").agg({"v1": "sum"})